[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/unboxai/examples-gallery/blob/main/text-classification/demo-banking.ipynb)


# Text classification using a custom

This notebook illustrates how custom models can be upladed to the Unbox platform.

## Importing the modules

In [2]:
import os
import fasttext
import numpy as np

## Training two fastText models on two datasets

In [3]:
rec_model = fasttext.train_supervised(input="rec.txt", lr=0.3, epoch=100, loss='hs')

Read 2M words
Number of words:  211125
Number of labels: 2
Progress: 100.0% words/sec/thread: 6976027 lr:  0.000000 avg.loss:  0.394708 ETA:   0h 0m 0s


In [4]:
rec_type_model = fasttext.train_supervised(input="rec_type.txt", lr=0.3, epoch=100, loss='hs')

Read 0M words
Number of words:  49236
Number of labels: 4
Progress: 100.0% words/sec/thread: 6655753 lr:  0.000000 avg.loss:  1.088968 ETA:   0h 0m 0s


In [5]:
rec_model.predict(["science and politics"]), rec_type_model.predict(["science and politics"])

(([['__label__not.rec']], [array([0.9151641], dtype=float32)]),
 ([['__label__rec.autos']], [array([0.5754844], dtype=float32)]))

In [6]:
rec_class_names = [s.replace("__label__", "") for s in rec_model.labels]
rec_type_class_names = [s.replace("__label__", "") for s in rec_type_model.labels]
rec_k = len(rec_class_names)
rec_type_k = len(rec_type_class_names)

In [7]:
rec_k, rec_type_k

(2, 4)

In [8]:
rec_class_names, rec_type_class_names

(['not.rec', 'rec'],
 ['rec.sport.hockey', 'rec.motorcycles', 'rec.sport.baseball', 'rec.autos'])

In [9]:
all_class_names = ['not.rec', 'rec.sport.hockey', 'rec.motorcycles', 'rec.sport.baseball', 'rec.autos']

In [10]:
def get_predictions(models, text_list):
    
    rec_model, rec_type_model = models
    
    predictions = rec_model.predict(text_list, k=rec_k)
    x, y = predictions
    
    predictions = rec_type_model.predict(text_list, k=rec_k)
    x2, y2 = predictions
    
    probabilities_full_list = []
    for rec_label_list, rec_prob_list, rec_type_label_list, rec_type_prob_list in zip(x, y, x2, y2):
        rec_label_prob_pair_dict = {}
        rec_type_label_prob_pair_dict = {}
        
        for rec_lbl, rec_prob, rec_type_lbl, rec_type_prob in zip(
            rec_label_list, rec_prob_list, rec_type_label_list, rec_type_prob_list
        ):
            rec_label_prob_pair_dict[
                rec_lbl.replace("__label__", "")
            ] = rec_prob
            rec_type_label_prob_pair_dict[
                rec_type_lbl.replace("__label__", "")
            ] = rec_type_prob
        
        rec_weight = rec_label_prob_pair_dict['rec']
        not_rec_weight = rec_label_prob_pair_dict['not.rec']
        probabilities_list = []
        
        if rec_weight > not_rec_weight:
            for cls in all_class_names:
                if cls in rec_type_label_prob_pair_dict:
                    p = rec_type_label_prob_pair_dict[cls]
                    probabilities_list.append(p)
                else:
                    probabilities_list.append(0.0)
        
        if not_rec_weight > rec_weight:
            for cls in all_class_names:
                if cls == 'not.rec':
                    p = rec_label_prob_pair_dict[cls]
                    probabilities_list.append(p)
                elif cls in rec_type_label_prob_pair_dict:
                    p = rec_type_label_prob_pair_dict[cls]*rec_weight
                    probabilities_list.append(p)
                else:
                    probabilities_list.append(0.0)
                    
        probabilities_full_list.append(probabilities_list)
        
        
    return np.array(probabilities_full_list)

In [11]:
phrases = ["science", "motorcycles are cool"]

In [12]:
get_predictions((rec_model, rec_type_model), phrases)

array([[0.99033558, 0.00254227, 0.00263809, 0.        , 0.        ],
       [0.        , 0.24073192, 0.        , 0.37864381, 0.        ]])

In [ ]:
all_class_names

In [ ]:
folder_name = "./dependencies"
rec_model_path = f"{folder_name}/rec_model.bin"
rec_type_model_path = f"{folder_name}/rec_type_model.bin"

# Save model binaries for packaging
os.makedirs(folder_name)
rec_model.save_model(rec_model_path)
rec_type_model.save_model(rec_type_model_path)

# Unbox

In [ ]:
import unboxapi
from unboxapi.models import ModelType
from unboxapi.tasks import TaskType

client = unboxapi.UnboxClient("YOUR_API_KEY_HERE")

In [ ]:
# Comment this out and uncomment the next section to load the project
project = client.create_project(
    name="Fasttext Stacked Model",
    description="Project to Demo Fasttext Stacked"
)

# Use this for loading the project on subsequent runs
'''
project = client.load_project(
    name="Fasttext Stacked Model"
)
'''

In [ ]:
custom_model_code = f"""
import fasttext

rec_model = fasttext.load_model("{rec_model_path}")
rec_type_model = fasttext.load_model("{rec_type_model_path}")
model = (rec_model, rec_type_model)
"""

ml_model = project.add_model(
    function=get_predictions, 
    model=None,
    dependent_dir=folder_name,
    custom_model_code=custom_model_code,
    model_type=ModelType.custom,
    task_type=TaskType.TextClassification,
    class_names=all_class_names,
    requirements_txt_file="./requirements.txt",
    name='Stacked model',
    description='there are two models here'
)
ml_model.to_dict()